In [1]:
import sys
import cornac
import tensorflow as tf

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")
print(f"Tensorflow version: {tf.__version__}")

SEED = 20240514
VERBOSE = True

System version: 3.11.8 (main, Feb 26 2024, 15:36:12) [Clang 14.0.6 ]
Cornac version: 2.1
Tensorflow version: 2.16.1


In [6]:
import cornac
import pandas as pd
from cornac.eval_methods import RatioSplit
from cornac.models import BPR
from cornac.metrics import Precision, Recall

df = pd.read_csv("./cs608_ip_train_v3.csv")
df1 = pd.read_csv("./cs608_ip_probe_v3.csv")
data_train = df.values.tolist()
data_val = df1.values.tolist()
data = data_train + data_val

# Split the data into training, validation and testing sets
rs = RatioSplit(data=data, test_size=0.1, val_size=0.1, seed=SEED, verbose=VERBOSE)

# Instantiate Recall@100 for evaluation
rec50 = cornac.metrics.Recall(50)

# Instantiate a matrix factorization model (e.g., BPR)
bpr = BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 21124
Number of items = 29665
Number of ratings = 225055
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 21124
Number of items = 29665
Number of ratings = 27175
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 21124
Number of items = 29665
Number of ratings = 27216
---
Total users = 21124
Total items = 29665


In [7]:
len(data), len(data_train), len(data_val)

(281321, 188656, 92665)

In [8]:
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch

# Grid Search
gs_bpr = GridSearch(
    model=bpr,
    space=[
        Discrete(name="k", values=[5, 10, 50, 100, 500]),
        Discrete(name="learning_rate", values=[0.001, 0.05, 0.01, 0.1]),
        Discrete(name="lambda_reg", values=[0.001, 0.01, 0.1, 1.0]),
    ],
    metric=rec50,
    eval_method=rs,
)

# Random Search
rs_bpr = RandomSearch(
    model=bpr,
    space=[
        Discrete(name="k", values=[5, 10, 50, 100, 500]),
        Continuous(name="learning_rate", low=0.001, high=0.01),
        Discrete(name="lambda_reg", values=[0.001, 0.01, 0.1, 1.0]),
    ],
    metric=rec50,
    eval_method=rs,
    n_trails=20,
)

In [9]:
# Define the experiment
cornac.Experiment(
    eval_method=rs,
    models=[gs_bpr, rs_bpr],
    metrics=[rec50],
    user_based=False,
).run()

# Obtain the best params
print(gs_bpr.best_params)
print(rs_bpr.best_params)


[GridSearch_BPR] Training started!


KeyboardInterrupt: 

In [ ]:
experiment.result, experiment.val_result

In [ ]:
%%time
from tqdm.notebook import tqdm

import zipfile

# Sort the DataFrame by 'user_id' in ascending order
df_sorted = df.sort_values(by='user_id')

# Extract unique user IDs sorted in ascending order
user_ids = df_sorted['user_id'].unique()

# Generate recommendations for all users
top_k_recommendations = []
for user_id in tqdm(user_ids):
    recs = bpr.recommend(user_id=user_id, k=50)
    top_k_recommendations.append(recs)

with open("submission.txt", "w") as file:
    for user_recommendations in top_k_recommendations:
        file.write(" ".join(map(str, user_recommendations)) + "\n")

# zip the submission file
with zipfile.ZipFile('submission.zip', 'w') as file:
    file.write('submission.txt')